# COMP 421 Fall 2022 Assignment 2

<img src="https://comp421.cs.unc.edu/static/images/COMP421Logo.png" 
     align="right" style="display:inline; width:100px; ">

Answer each question below by filling in code at the point indicated.
Do not modify any cells that say <font color="red">"You cannot and should not update this cell"!</font>

You __<font color="red">must</font>__ run the cells in order. It looks like you can come back later after restarting and pick up where you left off but that is not the case. When restarting you always have to start at the top of the notebook and run each cell in turn. 

The data supplied to you in this assignment will be graded with slightly different data.
For example, if a question asks you to create a list of names from the 
database and your code simply sets the list to the answers without 
querying the database, your answer will not be accepted by the grader.

In [1]:
# You cannot and should not update this cell
# setup
%autosave 15
import sqlite3
conn = sqlite3.connect('basketball.sqlite')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON')

import comp421
check, report = comp421.start('A2')

Autosaving every 15 seconds


In [2]:
# fill in your info here
Author = "mkrishn"
Collaborators = ["sarsimha"]

In [3]:
# You cannot and should not update this cell
check('Author', Author != 'youronyen', points=1, sort=True)
check('Collaborators', Collaborators != ['list', 'their', 'onyens', 'here'], points=1, sort=True)

Author appears correct
Collaborators appears correct


<img src="https://www.kaggle.com/static/images/site-logo.svg" width="200" style="float: right"/>

The database we'll be using in `A2-basketball.sqlite`.
It is my attempt to get as much information from college basketball in one place.
The information was taken from [kaggle.com college-basketball-dataset](https://www.kaggle.com/datasets/andrewsundberg/college-basketball-dataset) which
is a sports statistics and analytics website with no direct affiliation to the NCAA, its member conferences, or its member teams.

<p/>
If there are mistakes in the data, it's all my fault.
This is **not** an official database.

This database has the following schema:
```
CREATE TABLE Locations
    (location_id INTEGER PRIMARY KEY AUTOINCREMENT,
     city TEXT,
     state TEXT,
     population INTEGER,
     UNIQUE (city, state))

CREATE TABLE Stadiums
   (stadium_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    location_id REFERENCES Locations(location_id),
    capacity INTEGER,
    year_opened INTEGER)

CREATE TABLE Conferences
   (conference_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    short_name TEXT,
    year_founded INTEGER,
    number_mens_sports INTEGER UNSIGNED,
    number_womens_sports INTEGER UNSIGNED,
    location_id REFERENCES Locations(location_id), -- headquarter location
    UNIQUE(name))

CREATE TABLE Teams
   (team_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    conference_id REFERENCES Conferences(conference_id),
    stadium_id REFERENCES Stadiums(stadium_id),
    games_played INTEGER, -- # games played in 2020
    games_won INTEGER,    -- # games won in 2020
    power_rating FLOAT,   -- # Power Rating (Chance of beating an average Division I team)
    field_goal_percentage FLOAT,
    field_goal_percentage_allowed FLOAT,
    turn_over_rate FLOAT,
    steal_rate FLOAT,
    offensive_rebound_rate FLOAT,
    offensive_rebound_rate_allowed FLOAT,
    two_point_percentage FLOAT,
    two_point_allowed_percentage FLOAT,
    three_point_percentage FLOAT,
    three_point_allowed_percentage FLOAT,
    wins_above_bubble FLOAT)
```
Write code in the cells below to answer each question.

## 1. Find the following using SQL queries

**1a** UNC is known the college basketball database as 'North Carolina'.

Set variable `q1a` equal to the stadium name used by UNC basketball.

In [4]:
# write your query as a single string here
a1a = """
SELECT Stadiums.name
FROM Stadiums, Teams
WHERE Stadiums.stadium_id = Teams.stadium_id AND Teams.name = 'North Carolina'"""

# code to execute your query, leave this alone
q1a = cursor.execute(a1a).fetchone()[0]
print(f'The stadium used for basketball for  North Carolina is {q1a}.')

The stadium used for basketball for  North Carolina is Dean Smith Center.


In [5]:
# you cannot and should not update this cell
check('1a', q1a, points=8)

1a appears correct


**1b**. According to the college basketball database, how many teams are in the conference have a short named 'ACC'?

Set the variable `q1b` equal to the number of teams in the conference named 'ACC'.

In [6]:
a1b = """
SELECT COUNT(Teams.conference_id)
FROM Teams, Conferences
WHERE Conferences.short_name = "ACC" AND Conferences.conference_id = Teams.conference_id
"""

# test code
q1b = cursor.execute(a1b).fetchone()[0]
print(f'According to the college basketball database there are {q1b} teams in the ACC basketball conference.')

According to the college basketball database there are 15 teams in the ACC basketball conference.


In [7]:
# you cannot and should not update this cell
check('1b', q1b, points=10)

1b appears correct


**1c**. Again, according to the college basketball database,
what are the school names of teams in the ACC?

Set the variable `q1c` equal to the result of a SQL query that returns a Python list of tuples where the first tuple element is the school name.  The Python list should be in alphabetical order according to school name.

In [8]:
a1c = """
SELECT Teams.name
FROM Teams, Conferences
WHERE Conferences.short_name = "ACC" AND Conferences.conference_id = Teams.conference_id
ORDER BY Teams.name
"""

# test
q1c = cursor.execute(a1c).fetchall()
for row in q1c:
    print(f'{row[0]} is in the ACC')

Boston College is in the ACC
Clemson is in the ACC
Duke is in the ACC
Florida St. is in the ACC
Georgia Tech is in the ACC
Louisville is in the ACC
Miami FL is in the ACC
North Carolina is in the ACC
North Carolina St. is in the ACC
Notre Dame is in the ACC
Pittsburgh is in the ACC
Syracuse is in the ACC
Virginia is in the ACC
Virginia Tech is in the ACC
Wake Forest is in the ACC


In [9]:
# you cannot and should not update this cell
check('1c', q1c, points=10)

1c appears correct


**1d**. What is the stadium name and seating capacity of the UNC basketball team?

Write a Python SQL query that returns the stadium name and stadium capacity for UNC and
assign it's output to `q1d`.

In [10]:
a1d = """
SELECT Stadiums.name, Stadiums.capacity
FROM Stadiums, Teams
WHERE Stadiums.stadium_id = Teams.stadium_id AND Teams.name = "North Carolina" 
"""
q1d = cursor.execute(a1d).fetchone()

print(f'UNC uses stadium {q1d[0]} with a capacity of {q1d[1]} for home basketball games.')

UNC uses stadium Dean Smith Center with a capacity of 21750 for home basketball games.


In [11]:
# You cannot and should not update this cell
check('1d', q1d, points=10)

1d appears correct


**1e**. By seating capacity, what is the largest arena in the ACC?

Write a Python SQL query that returns the school name, stadium name, and stadium capacity for the
school(s) in the ACC with the largest capacity basketball arena?

In [12]:
a1e = """
SELECT Teams.name, Stadiums.name, MAX(Stadiums.capacity)
FROM Stadiums, Teams, Conferences
WHERE Teams.stadium_id = Stadiums.stadium_id AND Conferences.conference_id = Teams.conference_id AND Conferences.short_name = "ACC";
"""
q1e = cursor.execute(a1e).fetchone()

print(f'The largest stadium capacity in the ACC is stadium {q1e[1]} used by {q1e[0]} with a capacity of {q1e[2]}')

The largest stadium capacity in the ACC is stadium JMA Wireless Dome used by Syracuse with a capacity of 35446


In [13]:
# You cannot and should not update this cell
check('1e', q1e, points=15)

1e appears correct


## 2. Fill in the body of the functions below.

**2a**. Fill in the code below to make this function return the school name in the `conf_name` parameter's conference that played the least number of games.
If multiple schools have the same minimum number of games played, return the school that played the least number of games and appears alphabetically last.

**INPUT**: The function will take a parameter `conf_name` which is the abbreviation for the conference name.<br />     
**OUTPUT**: The function will return a single string with the school name that of the school that played the least number of games that appears alphabetically last of all the teams that played that number of games for the conference


In [14]:
def f2a(conf_name):
    #your code here
    sql = '''
    SELECT Teams.name
    FROM Teams, Conferences
    WHERE Teams.games_played = (
    SELECT MIN(Teams.games_played)
    FROM Teams, Conferences
    WHERE Conferences.short_name = '{conf_name}'
    AND Conferences.conference_id = Teams.conference_id)
    AND Conferences.short_name = '{conf_name}' AND Teams.conference_id = Conferences.conference_id
    ORDER BY Teams.name DESC
    '''.format(conf_name = conf_name)
    
    results = cursor.execute(sql).fetchall()
    return results[0][0]

conf_name = 'ACC'
q2a = f2a(conf_name)
print(f'{q2a} played the least number of games in the {conf_name}')

Louisville played the least number of games in the ACC


In [15]:
# You cannot and should not update this cell
confs = ['A10', 'B10', 'B12', 'ACC', 'SB']
for index, conf_name in enumerate(confs):
    q2a = f2a(conf_name)
    print(f'{q2a} player the least number of games in the {conf_name} conference')
    check(f'2a-{index+1}', q2a, points=20/len(confs))

Fordham player the least number of games in the A10 conference
2a-1 appears correct
Northwestern player the least number of games in the B10 conference
2a-2 appears correct
Iowa St. player the least number of games in the B12 conference
2a-3 appears correct
Louisville player the least number of games in the ACC conference
2a-4 appears correct
Coastal Carolina player the least number of games in the SB conference
2a-5 appears correct


**2b.** Write a function identical to *f2a* except that it returns all the teams that played the least number of games in the conference along with the number of games they played

**INPUT:** A conference short name
**OUTPUT:** A Python list of the team name(s) that played the least number of games in the conference along with the number of games they played.

In [16]:
def f2b(conf_name):
    # your code here
    sql = '''
    SELECT Teams.name, Teams.games_played
    FROM Teams, Conferences
    WHERE Teams.games_played = (
    SELECT MIN(Teams.games_played)
    FROM Teams, Conferences
    WHERE Conferences.short_name = '{conf_name}'
    AND Conferences.conference_id = Teams.conference_id)
    AND Conferences.short_name = '{conf_name}' AND Teams.conference_id = Conferences.conference_id
    ORDER BY Teams.name DESC
    '''.format(conf_name = conf_name)
    
    results = cursor.execute(sql).fetchall()
    return results
    
conf_name = 'ACC'
week_number = 1
q2b = f2b(conf_name)
for name, games in q2b:
    print(f'{name} played {games} games in the {conf_name} conference')

Louisville played 20 games in the ACC conference
Boston College played 20 games in the ACC conference


In [17]:
# You cannot and should not update this cell
confs = ['A10', 'B10', 'B12', 'ACC', 'SB']
for index, conf_name in enumerate(confs):
    q2b = f2b(conf_name)
    for name, games in q2b:
        print(f'{name} played {games} games in the {conf_name} conference')
    print()
    check(f'2b-{index+1}', q2a, points=25/len(confs))

Fordham played 14 games in the A10 conference

2b-1 appears correct
Northwestern played 23 games in the B10 conference

2b-2 appears correct
Iowa St. played 24 games in the B12 conference
Baylor played 24 games in the B12 conference

2b-3 appears correct
Louisville played 20 games in the ACC conference
Boston College played 20 games in the ACC conference

2b-4 appears correct
Coastal Carolina played 19 games in the SB conference

2b-5 appears correct


# Done!

Now get your report, save, and submit your notebook. I recommend you restart the kernel, and run all, just to be sure things are going to work correctly when I run it.

In [18]:
# You cannot and should not update this cell
# This cell will produce a summary report of your assignment
assert Author != 'youronyen'
assert Collaborators != ['list', 'their', 'onyens', 'here']
report(Author, Collaborators)

  Collaborators: ['sarsimha']
Report for mkrishn
  17 of 17 appear correct, 100 of 100 points


## Done!
<img src="https://comp421.cs.unc.edu/static/images/restartAndClearOutput.png" width="300" style="float: right" />

Now go back, restart the kernel (menu <font color="green">Kernel</font>-><font color="green">Restart and Clear</font>), and then Shift-Enter your way through the notebook to run all the cells again so you an be sure all your code will work as you expect during grading.

## Saving your work
<img src="https://comp421.cs.unc.edu/static/images/saveAndCheckpoint.png" width="300" style="float: right" />

Now save your work by going to (menu <font color='green'>File</font>-><font color='green'>Save and Checkpoint</font>)

## Submit your notebook

After saving, restarting the kernel, and verifying your report,
you can submit your notebook.
Go to https://comp421.cs.unc.edu/notebook


Note that if you actually saved your work you should not see the leaving site message below.
If you do see the `Leave Site` warning, cancel and save your work again.
<br />
<img src="https://comp421.cs.unc.edu/static/images/leaveSite.png" width="300" style="float: left" />